<a href="https://colab.research.google.com/github/josepmatias/nos_reten-ao/blob/main/objectivos_reactiva_vers%C3%A3o%20ciclo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install pyxlsb

In [3]:
#nome dos ficheiros
versao_ciclo="V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb"
resumo_nome=".Resumo templates_abril24.xlsb"
nome_folha_objectivo="Objetivos reativa"

In [4]:
#fazer o upload dos ficheiros
from google.colab import files
uploaded = files.upload()
uploaded = files.upload()

Saving V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront.xlsb to V1_Tratar_Cycle_Detail_RRR__PQ_MASTER_Upfront (1).xlsb


Saving .Resumo templates_abril24.xlsb to .Resumo templates_abril24 (1).xlsb


In [5]:
#ler a dataframe da 1ª versão do ciclo

ciclo=pd.read_excel(versao_ciclo, sheet_name="Dados carregados").iloc[:, :7]

In [6]:
#formatar a dataframe da 1ª versão do ciclo
ciclo2=ciclo.drop(ciclo.iloc[:, 3].name, axis=1)

indice_max=ciclo2.isna().all(axis=1).idxmax()

ciclo3=ciclo2.iloc[:indice_max, :]

ciclo3.columns=["Tipo", "Objective name", "Objective value", "Tipo 2", "Objective name 2", "Objective value 2"]

ciclo3.head()


,Tipo,Objective name,Objective value,Tipo 2,Objective name 2,Objective value 2
0,Decisores,Exclusivos,1.5,Renegociação,Exclusivos,0.5
1,Decisores,MATRIZ BASE MTR,1.5,Renegociação,Convergente_MATRIZ BASE 2LO,0.0
2,Decisores,MATRIZ BASE C2C,1.5,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...,0.0
3,Decisores,MATRIZ BASE 2LO,1.5,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...,0.0
4,Decisores,MATRIZ BASE OUT,1.5,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO,0.0


In [11]:
#dividir dataframe no ciclo em 2 - ciclo_half1 inclui "decisores" e "retenção" - ciclo_half2 inclui "renegociação" - não são extraídas as colunas "Objective value"
ciclo_half1=ciclo3.iloc[:, :2]

ciclo_half2=ciclo3.iloc[:, 3:5]


In [10]:
#inspeccionar dataframe "ciclo_half1"
ciclo_half1

,Tipo,Objective name
0,Decisores,Exclusivos
1,Decisores,MATRIZ BASE MTR
2,Decisores,MATRIZ BASE C2C
3,Decisores,MATRIZ BASE 2LO
4,Decisores,MATRIZ BASE OUT
...,...,...
64,Retenção,Não Convergente_DTH_AT_TM OUT
65,Retenção,Não Convergente_Potenciais UpSellers_AT_TM OUT
66,Retenção,Convergente_AT_TM OUT
67,NaN,Convergente_EXCLUSIVOS


In [12]:
#inspeccionar dataframe "ciclo_half2"
ciclo_half2

,Tipo 2,Objective name 2
0,Renegociação,Exclusivos
1,Renegociação,Convergente_MATRIZ BASE 2LO
2,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...
3,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...
4,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO
...,...,...
64,NaN,NaN
65,NaN,NaN
66,NaN,NaN
67,NaN,NaN


In [14]:
#inspeccionar "ciclo_half2" - remover n/a na dataframe "ciclo_half2"
ciclo_half2=ciclo_half2.iloc[:ciclo_half2.isna().all(axis=1).idxmax(), :]

print(ciclo_half2.shape)
ciclo_half2.head()

(58, 2)


,Tipo 2,Objective name 2
0,Renegociação,Exclusivos
1,Renegociação,Convergente_MATRIZ BASE 2LO
2,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...
3,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...
4,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO


In [15]:
#substituir n/a por vazio na coluna "tipo 2"
print(ciclo_half2.isna().sum())

ciclo_half22=ciclo_half2.fillna("")

ciclo_half22.isna().sum()

Tipo 2              2
Objective name 2    0
dtype: int64


Tipo 2              0
Objective name 2    0
dtype: int64

In [16]:
#função "chave" para criar 1 chave entre a coluna "tipo" e "objective name"
def chave(db, col1, col2):
  lista1=db[col1].tolist()
  lista2=db[col2].tolist()

  lista=list(map(lambda i, j: i + "_" + j if len(i) else j, lista1, lista2))

  return lista

lista_half2=chave(ciclo_half22, "Tipo 2", "Objective name 2")

In [75]:
#adicionar chave á dataframe "ciclo_half22"

ciclo_half22["chave"]=lista_half2

In [18]:
#ler ficheiro ".resumo templates"
resumo=pd.read_excel(resumo_nome, sheet_name=nome_folha_objectivo).iloc[:, :6]

resumo2=resumo.dropna()

print(resumo2.shape)
resumo2.head()

(122, 6)


,SFID,Type,Objective Name,Begin Date,End Date,Objective\n(Value)
0,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Qualquer_Qualquer...,45383,45412.0,0.5
1,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Convergente_Qualq...,45383,45412.0,0.0
2,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_C...,45383,45412.0,0.0
3,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_C...,45383,45412.0,0.0
4,RETENÇÃO RESIDENCIAL,Group,RRR_GCO_04.2024_Renegociação_Não Convergente_Q...,45383,45412.0,0.0


In [28]:
#formatar ficheiro ".resumo templates" - substituir caracteres (ex: RRR_GCO_04.2024_)

caracter_subs=resumo2.iloc[:, 2].apply(lambda i: "_".join(i.split("_")[:3]) + "_").values[0]

caracter_subs
resumo2["Objective Name"]=resumo2["Objective Name"].replace(regex=caracter_subs + "*", value="")

resumo2.head()


<ipython-input-28-4a52c3956e64>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumo2["Objective Name"]=resumo2["Objective Name"].replace(regex=caracter_subs + "*", value="")


,SFID,Type,Objective Name,Begin Date,End Date,Objective\n(Value),Objective Name 2
0,RETENÇÃO RESIDENCIAL,Group,Qualquer_EXCLUSIVOS,45383,45412.0,0.5,Renegociação_EXCLUSIVOS
1,RETENÇÃO RESIDENCIAL,Group,Renegociação_Convergente_Qualquer_Qualquer_MAT...,45383,45412.0,0.0,Renegociação_Convergente_MATRIZ BASE 2LO
2,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Com CM_3P Alto Va...,45383,45412.0,0.0,Renegociação_Não Convergente_Com CM_3P Alto Va...
3,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Com CM_3P Baixo V...,45383,45412.0,0.0,Renegociação_Não Convergente_Com CM_3P Baixo V...
4,RETENÇÃO RESIDENCIAL,Group,Renegociação_Não Convergente_Qualquer_DTH_MATR...,45383,45412.0,0.0,Renegociação_Não Convergente_DTH_MATRIZ BASE 2LO


In [30]:
#verificar se existem n/a na dataframe "resumo2"
resumo2.isna().sum()

SFID                  0
Type                  0
Objective Name        0
Begin Date            0
End Date              0
Objective\n(Value)    0
Objective Name 2      0
dtype: int64

In [31]:
#substituir a substring "Qualquer_" na dataframe "resumo2"
resumo2["Objective Name 2"]=resumo2["Objective Name"].str.replace("Qualquer_", "", regex=False)

<ipython-input-31-356aae1a2f1e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumo2["Objective Name 2"]=resumo2["Objective Name"].str.replace("Qualquer_", "", regex=False)


In [32]:
#inspeccionar a dataframe "resumo2" após alterações
resumo2["Objective Name 2"]

0                                             EXCLUSIVOS
1               Renegociação_Convergente_MATRIZ BASE 2LO
2      Renegociação_Não Convergente_Com CM_3P Alto Va...
3      Renegociação_Não Convergente_Com CM_3P Baixo V...
4       Renegociação_Não Convergente_DTH_MATRIZ BASE 2LO
                             ...                        
117    Retenção_Não Convergente_Sem CM_3P Baixo Valor...
118               Retenção_Não Convergente_DTH_AT_TM 2LO
119    Retenção_Não Convergente_Potenciais UpSellers_...
120                       Retenção_Convergente_AT_TM 2LO
121                                  Decisores_AT_TM 2LO
Name: Objective Name 2, Length: 122, dtype: object

In [57]:
#verificar se há linhas na dataframe "resumo" que não estão na dataframe ciclo
print("linhas comuns entre versão ciclo (lista_half2) e versão resumo")
print([i for i in resumo2.loc[resumo2["Objective Name 2"].str.contains("renegociação", case=False), "Objective Name 2"].tolist() if i in lista_half2])
print("-----------------------------------------------------------------------------")
print("linhas na versão resumo que não estão na versão do ciclo")
print([i for i in resumo2.loc[resumo2["Objective Name 2"].str.contains("renegociação", case=False), "Objective Name 2"].tolist() if i not in lista_half2])

print("------------------------------------------------------------------------------")
print("linhas na versão ciclo que não estão na versão resumo")
print([i for i in lista_half2 if i not in resumo2.loc[resumo2["Objective Name 2"].str.contains("renegociação", case=False), "Objective Name 2"].tolist()])
#print(len(lista_half2))

#print(len(resumo2.loc[resumo2["Objective Name 2"].str.contains("renegociação", case=False), "Objective Name 2"]))

linhas comuns entre versão ciclo (lista_half2) e versão resumo
['Renegociação_Convergente_MATRIZ BASE 2LO', 'Renegociação_Não Convergente_Com CM_3P Alto Valor_MATRIZ BASE 2LO', 'Renegociação_Não Convergente_Com CM_3P Baixo Valor_MATRIZ BASE 2LO', 'Renegociação_Não Convergente_DTH_MATRIZ BASE 2LO', 'Renegociação_Não Convergente_Potenciais UpSellers_MATRIZ BASE 2LO', 'Renegociação_Não Convergente_Sem CM_3P Alto Valor_MATRIZ BASE 2LO', 'Renegociação_Não Convergente_Sem CM_3P Baixo Valor_MATRIZ BASE 2LO', 'Renegociação_Convergente_MATRIZ BASE C2C', 'Renegociação_Não Convergente_Com CM_3P Alto Valor_MATRIZ BASE C2C', 'Renegociação_Não Convergente_Com CM_3P Baixo Valor_MATRIZ BASE C2C', 'Renegociação_Não Convergente_DTH_MATRIZ BASE C2C', 'Renegociação_Não Convergente_Potenciais UpSellers_MATRIZ BASE C2C', 'Renegociação_Não Convergente_Sem CM_3P Alto Valor_MATRIZ BASE C2C', 'Renegociação_Não Convergente_Sem CM_3P Baixo Valor_MATRIZ BASE C2C', 'Renegociação_Convergente_MATRIZ BASE IN', 'Renego

In [58]:
resumo2[resumo2["Objective Name 2"].str.contains("exclusivo", case=False)]

,SFID,Type,Objective Name,Begin Date,End Date,Objective\n(Value),Objective Name 2
0,RETENÇÃO RESIDENCIAL,Group,Qualquer_EXCLUSIVOS,45383,45412.0,0.50,EXCLUSIVOS
32,RETENÇÃO RESIDENCIAL,Group,Retenção_Qualquer_Qualquer_Qualquer_EXCLUSIVOS,45383,45412.0,0.89,Retenção_EXCLUSIVOS
98,RETENÇÃO RESIDENCIAL,Group,Decisores_Qualquer_Qualquer_Qualquer_EXCLUSIVOS,45383,45412.0,1.50,Decisores_EXCLUSIVOS


In [86]:
#preenchimento da coluna "Objectivo valor" com os novos valores do resumo
def pontuacao(db, db2, lista, str):
  #ir buscar pontuação à versão resumo
  for i, j in zip(db.loc[db["Objective Name 2"].isin(lista), "Objective Name 2"], db.loc[db["Objective Name 2"].isin(lista), "Objective\n(Value)"]):
    #print(j)
    indice=db2[db2["chave"]==i].index
    db2.loc[indice, "Objectivo valor"]=j
  #acrescentar valores de exclusivo
  valores_exclusivo=db.loc[db["Objective Name 2"]==str,  "Objective\n(Value)"].values[0]
  db2.loc[db2["Objectivo valor"].isna()==True, "Objectivo valor"]=valores_exclusivo

  #remover coluna "chave"
  db2.drop("chave", axis=1)

  return db2

pontuacao(resumo2, ciclo_half22, lista=lista_half2, str="EXCLUSIVOS")

,Tipo 2,Objective name 2,chave,Objectivo valor
0,Renegociação,Exclusivos,Renegociação_Exclusivos,0.50
1,Renegociação,Convergente_MATRIZ BASE 2LO,Renegociação_Convergente_MATRIZ BASE 2LO,0.00
2,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...,Renegociação_Não Convergente_Com CM_3P Alto Va...,0.00
3,Renegociação,Não Convergente_Com CM_3P Baixo Valor_MATRIZ B...,Renegociação_Não Convergente_Com CM_3P Baixo V...,0.00
4,Renegociação,Não Convergente_DTH_MATRIZ BASE 2LO,Renegociação_Não Convergente_DTH_MATRIZ BASE 2LO,0.00
5,Renegociação,Não Convergente_Potenciais UpSellers_MATRIZ BA...,Renegociação_Não Convergente_Potenciais UpSell...,0.00
6,Renegociação,Não Convergente_Sem CM_3P Alto Valor_MATRIZ BA...,Renegociação_Não Convergente_Sem CM_3P Alto Va...,0.00
7,Renegociação,Não Convergente_Sem CM_3P Baixo Valor_MATRIZ B...,Renegociação_Não Convergente_Sem CM_3P Baixo V...,0.00
8,Renegociação,Convergente_MATRIZ BASE C2C,Renegociação_Convergente_MATRIZ BASE C2C,0.41
9,Renegociação,Não Convergente_Com CM_3P Alto Valor_MATRIZ BA...,Renegociação_Não Convergente_Com CM_3P Alto Va...,0.00


In [92]:
ciclo_half22=ciclo_half22.drop(ciclo_half22.iloc[:, 2].name, axis=1)
ciclo_half22.iloc[:, 2].sum()

9.07